In [10]:
import datetime

import pandas as pd

from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import StrategyType, TradeRouting, ReserveCurrency


# Tell what trade execution engine version this strategy needs to use
trading_strategy_engine_version = "0.1"


# The duration of the backtesting period
START_AT_DATA = datetime.datetime(2023, 1, 1)
START_AT = datetime.datetime(2023, 2, 1)
END_AT = datetime.datetime(2023, 11, 8)
CANDLE_TIME_BUCKET = TimeBucket.h4

In [2]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /home/aarnihuuskonen/.tradingstrategy


In [5]:
from tradeexecutor.utils.binance import create_binance_universe

universe = create_binance_universe(
    symbols=["ETHUSDT"],
    candle_time_bucket=CANDLE_TIME_BUCKET,
    start_at=START_AT_DATA,
    end_at=END_AT,
    include_lending=False
)

print(f"We loaded {universe.universe.candles.get_candle_count():,} candles.")

Loaded candlestick data for ETHUSDT from cache:   0%|          | 0/1 [00:00<?, ?it/s]

We loaded 1,867 candles.


In [11]:
universe.data_universe.candles.df.head()

,open,high,low,close,volume,pair_id,base_token_symbol,quote_token_symbol,exchange_slug,chain_id,...,buy_volume_all_time,address,exchange_id,token0_address,token1_address,token0_symbol,token1_symbol,token0_decimals,token1_decimals,timestamp
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,1196.13,1196.74,1191.71,1194.04,20292.0602,1,ETH,USDT,binance,-1.0,...,0.0,0xe82ac67166a910f4092c23f781cd39e46582ec9c,129875571.0,0x4b2d72c1cb89c0b2b320c43bb67ff79f562f5ff4,0x5b1a1833b16b6594f92daa9f6d9b7a6024bce9d0,ETH,USDT,18.0,18.0,2023-01-01 00:00:00
2023-01-01 04:00:00,1194.05,1195.75,1190.57,1195.06,18654.4594,1,ETH,USDT,binance,-1.0,...,0.0,0xe82ac67166a910f4092c23f781cd39e46582ec9c,129875571.0,0x4b2d72c1cb89c0b2b320c43bb67ff79f562f5ff4,0x5b1a1833b16b6594f92daa9f6d9b7a6024bce9d0,ETH,USDT,18.0,18.0,2023-01-01 04:00:00
2023-01-01 08:00:00,1195.07,1197.41,1193.98,1196.34,10749.1851,1,ETH,USDT,binance,-1.0,...,0.0,0xe82ac67166a910f4092c23f781cd39e46582ec9c,129875571.0,0x4b2d72c1cb89c0b2b320c43bb67ff79f562f5ff4,0x5b1a1833b16b6594f92daa9f6d9b7a6024bce9d0,ETH,USDT,18.0,18.0,2023-01-01 08:00:00
2023-01-01 12:00:00,1196.35,1198.50,1193.54,1196.81,15490.8573,1,ETH,USDT,binance,-1.0,...,0.0,0xe82ac67166a910f4092c23f781cd39e46582ec9c,129875571.0,0x4b2d72c1cb89c0b2b320c43bb67ff79f562f5ff4,0x5b1a1833b16b6594f92daa9f6d9b7a6024bce9d0,ETH,USDT,18.0,18.0,2023-01-01 12:00:00
2023-01-01 16:00:00,1196.81,1204.51,1195.88,1201.78,31016.5151,1,ETH,USDT,binance,-1.0,...,0.0,0xe82ac67166a910f4092c23f781cd39e46582ec9c,129875571.0,0x4b2d72c1cb89c0b2b320c43bb67ff79f562f5ff4,0x5b1a1833b16b6594f92daa9f6d9b7a6024bce9d0,ETH,USDT,18.0,18.0,2023-01-01 16:00:00


In [20]:
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

df = universe.data_universe.candles.df

candlesticks = go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    showlegend=False
)


volume_bars = go.Bar(
    x=df.index,
    y=df['volume'],
    showlegend=False
)

fig = make_subplots(rows=2,
cols=1,
shared_xaxes=True,
vertical_spacing=0.02)

fig.add_trace(candlesticks, row=1, col=1)

fig.add_trace(volume_bars, row=2, col=1)

fig.show()


In [21]:
def calculate_volume_bars(df: pd.DataFrame, volume_threshold: float):

    volume_data = df["volume"]

    cumulative_volume = 0

    idx = []

    for i, x in enumerate(volume_data):
        cumulative_volume += x
        if cumulative_volume >= volume_threshold:
            idx.append(i)
            cumulative_volume = 0
    return df.iloc[idx]